In [10]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating project at `~/Robotics/SparseLDL/julia`


In [11]:
using LinearAlgebra
using BlockDiagonals

In [12]:
#Let's make up a linear system
n = 4
m = 2
dt = 0.1
A = [I(2) dt*I(2); zeros(2,2) I(2)];
B = [0.5*dt*dt*I(2); dt*I(2)];

In [13]:
#LQR Cost Matrices
Q = Diagonal([1; 1; 0.1; 0.1]);
R = Diagonal([0.3; 0.3]);

In [14]:
#Permuted KKT Matrix
#We stack z = [x1; u1; λ2; x2; u2; λ3; x3]
#and g = [∇x1L; ∇u1L; A*x_1+B*u_1-x_2; ∇x2L; ∇u2L; A*x_2+B*u_2-x_3]
KKT = [Q zeros(n,m) A' zeros(n,n) zeros(n,m+n+n);
       zeros(m,n) R B' zeros(m,n) zeros(m,m+n+n);
       A B zeros(n,n) -I(n)       zeros(n,m+n+n);
       zeros(n,n) zeros(n,m) -I(n) Q zeros(n,m) A' zeros(n,n);
       zeros(m,n+m+n+n) R B' zeros(m,n);
       zeros(n,n+m+n) A B zeros(n,n) -I(n);
       zeros(n,n+m+n+n+m) -I(n) Q];

In [23]:
D1 = Matrix(Q)
D2 = Matrix(R)
D3 = -A*inv(D1)*A'-B*inv(D2)*B'
D4 = Q-inv(D3)
D5 = R
D6 = -A*inv(D4)*A'-B*inv(D5)*B'
D7 = Q-inv(D6)
D = BlockDiagonal([D1, D2, D3, D4, D5, D6, D7]);
D3

4×4 Matrix{Float64}:
 -1.10008   0.0       -1.00167    0.0
  0.0      -1.10008    0.0       -1.00167
 -1.00167   0.0      -10.0333     0.0
  0.0      -1.00167    0.0      -10.0333

In [16]:
L = [I(n) zeros(n,m+n+n) zeros(n,m+n+n);
     zeros(m,n) I(m) zeros(m,n+n) zeros(m,m+n+n);
     A*inv(D1) B*inv(D2) I(n) zeros(n,n) zeros(n,m+n+n);
     zeros(n,n) zeros(n,m) -inv(D3) I(n) zeros(n,m+n+n);
     zeros(m,n+m+n+n) I(m) zeros(m,n+n);
     zeros(n,n+m+n) A*inv(D4) B*inv(D5) I(n) zeros(n,n);
     zeros(n,n+m+n+n+m) -inv(D6) I(n)];

In [17]:
maximum(abs.(L*D*L'-KKT))

2.220446049250313e-16